#Initialising nvcc

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zfnj58xw
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zfnj58xw
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4305 sha256=590a25fdf3cacc2002712ef35b225f2c5de05bd40daccf39edb5c95d4e49c784
  Stored in directory: /tmp/pip-ephem-wheel-cache-phg1k0zl/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
!nvidia-smi

Wed Jun 23 22:48:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Generating Random numbers using curand api

In [ ]:
# Generating Random numbers using curand api

%%cu
#include <unistd.h>
#include <stdio.h>
#include<iostream>
#include<fstream>
#include<cmath>
using namespace std;

/* we need these includes for CUDA's random number stuff */
#include <curand.h>
#include <curand_kernel.h>

#define N 1000000

#define T 512
#define B (N/T)+1
#define MAX 599

__global__ void init(unsigned int seed, curandState_t* states) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  /* we have to initialize the state */
  curand_init(seed, 
              i, 
              0, 
              &states[i]);
}


__global__ void randoms(curandState_t* states, unsigned int* numbers) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  numbers[i] = curand(&states[i]) % MAX;
}

int main() {
  curandState_t* states;
  ofstream fout;
  fout.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::out);
  
  cudaMalloc((void**) &states, N * sizeof(curandState_t));
  init<<<B, T>>>(time(0), states);

  unsigned int* cpu_nums = new unsigned int[N];
  unsigned int* gpu_nums;
  cudaMalloc((void**) &gpu_nums, N * sizeof(unsigned int));

  /* invoke the kernel to get some random numbers */
  randoms<<<B, T>>>(states, gpu_nums);

  /* copy the random numbers back */
  cudaMemcpy(cpu_nums, gpu_nums, N * sizeof(unsigned int), cudaMemcpyDeviceToHost);

  /* print them out */
  int y = sqrt(N);
  for (int i = 0; i < y; i++) {
      for(int j=0;j<y;j++){
          printf("%u\t", (cpu_nums[i*y+j]%MAX));
          fout<<(cpu_nums[i*y+j]%MAX)<<"\t";
      }
      cout<<"\n";
    fout<<"\n";
  }

  /* free the memory we allocated for the states and numbers */
  cudaFree(states);
  cudaFree(gpu_nums);
  fout.close();
  return 0;
}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#matrix multiplication using shared memory

In [ ]:
# matrix multiplication using shared memory

%%cu
#include<stdio.h>
#include<iostream>
#include<fstream>
#include<cuda.h>
#include <chrono> 
using namespace std;

#define row1 10 
#define col1 10 
#define row2 10
#define col2 10

__global__ void matproductsharedmemory(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    __shared__ int p[col1];

    int i;
    int k=threadIdx.x;

    n[col2*y+x]=0;

   p[k]=l[col1*y+k]*m[col2*k+x];

  __syncthreads();

  for(i=0;i<col1;i++)
  n[col2*y+x]=n[col2*y+x]+p[i];
}

int main()
{
    std::chrono::time_point<std::chrono::system_clock> start, end; 
 
    ifstream fin;
    fin.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::in);
 
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int seq[row1][col2];
    int *d,*e,*f;
    int i,j,k;

    printf("\n Entering elements of first matrix of size 10*10\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                fin>>a[i][j];
                printf("%d\t",a[i][j]);
            }
     printf("\n");
    }
    printf("\n Enter elements of second matrix of size 10*10\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    fin>>b[i][j];
                 printf("%d\t",b[i][j]);
                }
         printf("\n");
        }

  start = std::chrono::system_clock::now();
 
   cudaMalloc((void **)&d,row1*col1*sizeof(int));
    cudaMalloc((void **)&e,row2*col2*sizeof(int));
    cudaMalloc((void **)&f,row1*col2*sizeof(int));

 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice);

dim3 grid(col2,row1);

matproductsharedmemory<<<grid,col1>>>(d,e,f);

 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost);
  
  end = std::chrono::system_clock::now();
 
 printf("\n Product of two matrices using parallel execution:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }
    
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nParallel algorithm elapsed time: " << elapsed_seconds.count() << "s\n"; 
    
    start = std::chrono::system_clock::now();
    for(i=0;i<row1;i++){
        for(j=0;j<col2;j++){
            seq[i][j] = 0;
            for(k=0;k<col1;k++){
                seq[i][j] += a[i][k] * b[k][j];
            }
        }
    }
    end = std::chrono::system_clock::now();
 
    printf("\n Product of two matrices using sequential execution:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",seq[i][j]);
        }
        printf("\n");
    }

   elapsed_seconds = end - start;
    cout<< "\n Sequential algorithm elapsed time: " << elapsed_seconds.count() << "s\n"; 
 
    printf("\n Error matrix:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",seq[i][j] - c[i][j]);
        }
        printf("\n");
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
    fin.close();
    return 0;
}


 Entering elements of first matrix of size 10*10
311	525	326	198	288	53	545	147	441	164	
155	111	46	8	349	360	287	411	159	1	
462	65	361	542	576	430	493	287	164	39	
489	63	241	77	483	146	111	353	139	172	
379	497	369	291	572	477	311	1	501	218	
200	321	358	477	335	270	116	375	505	443	
29	77	12	490	71	141	567	512	411	120	
34	494	343	50	298	524	497	420	180	506	
55	122	302	243	197	400	436	166	242	520	
464	96	378	377	151	344	577	248	594	529	

 Enter elements of second matrix of size 10*10
200	230	333	34	113	358	177	105	565	136	
588	349	100	149	171	40	178	2	286	244	
306	308	401	485	432	11	336	351	350	520	
312	585	486	44	261	258	215	68	356	76	
118	1	57	135	410	13	3	142	360	395	
544	283	394	21	30	540	379	582	238	276	
209	464	435	116	575	559	171	3	267	225	
245	217	173	431	414	178	87	292	455	360	
244	265	180	277	248	347	253	93	206	583	
588	314	213	357	491	391	453	176	356	30	

 Product of two matrices using parallel execution:
949204	939420	797133	602896	1001223	767344	613403	347773	988377	920920	

#matrix multiplication without using shared memory

In [ ]:
# matrix multiplication without using shared memory

%%cu
#include<stdio.h>
#include<iostream>
#include<fstream>
#include<cuda.h>
#include <chrono> 
using namespace std;

#define row1 100 
#define col1 100
#define row2 100
#define col2 100

__global__ void matproduct(int *l,int *m, int *n)
{
    int x=blockIdx.x;
    int y=blockIdx.y;
    int k;
  
    n[col2*y+x]=0;
    for(k=0;k<col1;k++)
    {
      n[col2*y+x]=n[col2*y+x]+l[col1*y+k]*m[col2*k+x];
    }
}

int main()
{
    std::chrono::time_point<std::chrono::system_clock> start, end; 
 
    ifstream fin;
    fin.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::in);
 
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int *d,*e,*f;
    int i,j;

    printf("\n Entering elements of first matrix of size 10*10\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                fin>>a[i][j];
                printf("%d\t",a[i][j]);
            }
     printf("\n");
    }
    printf("\n Enter elements of second matrix of size 10*10\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    fin>>b[i][j];
                 printf("%d\t",b[i][j]);
                }
         printf("\n");
        }

  start = std::chrono::system_clock::now();
 
    cudaMalloc((void **)&d,row1*col1*sizeof(int));
    cudaMalloc((void **)&e,row2*col2*sizeof(int));
    cudaMalloc((void **)&f,row1*col2*sizeof(int));

 cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice);
 cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice);

dim3 grid(col2,row1);
/* Here we are defining two dimensional Grid(collection of blocks) structure. Syntax is dim3 grid(no. of columns,no. of rows) */

    matproduct<<<grid,1>>>(d,e,f);

 cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost);
  end = std::chrono::system_clock::now();
 
 printf("\n Product of two matrices using parallel execution:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
              if(c[i][j] < 10000000){
                  printf("\t");
              }
        }
        printf("\n");
    }
    
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nParallel algorithm elapsed time: " << elapsed_seconds.count() << "s\n"; 

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);

    return 0;
}



 Entering elements of first matrix of size 10*10
311	525	326	198	288	53	545	147	441	164	155	111	46	8	349	360	287	411	159	1	462	65	361	542	576	430	493	287	164	39	489	63	241	77	483	146	111	353	139	172	379	497	369	291	572	477	311	1	501	218	200	321	358	477	335	270	116	375	505	443	29	77	12	490	71	141	567	512	411	120	34	494	343	50	298	524	497	420	180	506	55	122	302	243	197	400	436	166	242	520	464	96	378	377	151	344	577	248	594	529	
200	230	333	34	113	358	177	105	565	136	588	349	100	149	171	40	178	2	286	244	306	308	401	485	432	11	336	351	350	520	312	585	486	44	261	258	215	68	356	76	118	1	57	135	410	13	3	142	360	395	544	283	394	21	30	540	379	582	238	276	209	464	435	116	575	559	171	3	267	225	245	217	173	431	414	178	87	292	455	360	244	265	180	277	248	347	253	93	206	583	588	314	213	357	491	391	453	176	356	30	
260	226	530	309	144	455	400	356	120	108	505	206	286	588	521	288	349	405	237	314	191	475	589	338	38	157	516	400	242	17	109	305	233	261	192	147	432	412	185	4	47	344	382	269	528	180	226	398	21

#Prims Algorithm

In [ ]:
%%cu
#include <stdio.h> 
#include <queue>
#include <set>
#include <list>
#include <iterator>
#include <algorithm>
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define ARRAY_SIZE 30
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}
struct distNode { 
	int node; 
	int dist; 
  bool operator<(const distNode& rhs) const
    {
        return dist > rhs.dist || (dist == rhs.dist && node > rhs.node);;
    }
}; 
struct edge { 
	int from; 
	int to; 
 	int weight;
   	bool operator<(const edge& rhs) const
     {
         return weight > rhs.weight || (weight == rhs.weight && to > rhs.to);
     }
};
struct fromTo { 
	int from; 
	int to; 
   	bool operator<(const fromTo& rhs) const
     {
         return to < rhs.to || (to == rhs.to && from < rhs.from);
     }
};

__device__ __managed__ int parent[ARRAY_SIZE]; // Array to store parent nodes
__device__ __managed__ int dist[ARRAY_SIZE]; // Array to store node distances
__device__ __managed__ bool fixed[ARRAY_SIZE]; // Array to store flags for node traversal
std::vector<bool> nonEmptyIndices; // Array to store non empty indices of vertices

std::priority_queue<distNode> H; //binary heap of (j,dist) initially empty;
__device__ __managed__ int Q[ARRAY_SIZE], R[ARRAY_SIZE]; //set of vertices initially empty;
__device__ __managed__ fromTo T[ARRAY_SIZE*ARRAY_SIZE]; //{ set of edges } initially {};
__device__ __managed__ fromTo mwe[ARRAY_SIZE*ARRAY_SIZE]; //set of edges; minimum weight edges for all vertices
__device__ __managed__ int z_device, Q_index=0, R_index=0, mwe_index=0, T_index=0; //Indices to synchronize between host & device
__device__ __managed__ int edge_cnt=0;

int allvertex_in[ARRAY_SIZE*ARRAY_SIZE], alledge_in[ARRAY_SIZE*ARRAY_SIZE], allweight_in[ARRAY_SIZE*ARRAY_SIZE];
class Graph
{
public:
	std::vector<std::vector<edge>> adjList;
	Graph(std::vector<edge> const &edges, int N)
	{
		adjList.resize(N);
		nonEmptyIndices.resize(N);
		for (auto &e: edges)
		{
			int from = e.from;
			int to = e.to;
			int weight = e.weight;
			adjList[from].push_back(edge{from, to, weight});
			adjList[to].push_back(edge{to, from, weight});
			nonEmptyIndices[from] = true;
			nonEmptyIndices[to] = true;
		}
	}
};

void printGraph(Graph const &graph)
{
	printf("Input Graph\n");
	for (int i = 0; i < graph.adjList.size(); i++)
	{
		for (edge v : graph.adjList[i]){
			printf("( %d, %d, %d )", v.from, v.to, v.weight);
		}
	}
}
void deleteElement(int arr[], int arr_index, int size) 
{

   if (arr_index < size) 
   { 
     size = size - 1; 
     for (int j=arr_index; j<size; j++) 
        arr[j] = arr[j+1]; 
   }
} 
__device__ bool ifExist(int arr[], int val){
		for (int i=0; i<ARRAY_SIZE; i++) {
				if (arr[i] == val)
					return true;
		}
		return false;
}

__device__ bool ifExistMWE(fromTo arr[], fromTo ft){
		for (int i=0; i<edge_cnt; i++) {
				if (arr[i].from == ft.from && arr[i].to == ft.to)
					return true;
		}
		return false;
}
void load_kernelArrays(Graph const &graph) {
	for (int i = 0; i < graph.adjList.size(); i++) {
		for(edge adj : graph.adjList[i]) {
			allvertex_in[edge_cnt] = adj.from;
			alledge_in[edge_cnt] = adj.to;
			allweight_in[edge_cnt] = adj.weight;
			edge_cnt++;
		}
	}
}
void initMWE(Graph const &graph) 
{ 
	for (int i = 0; i < graph.adjList.size(); i++) {
		int prevWeight=INT_MAX;
		int min_to, minFrom;
		for (auto it=graph.adjList[i].begin(); it!=graph.adjList[i].end(); it++) {
			edge adj = *it;
			if (adj.weight < prevWeight) { 
				min_to = adj.to;
				minFrom = adj.from;
				prevWeight = adj.weight;
			}
		} 
		mwe[mwe_index] = fromTo{minFrom, min_to};
		mwe_index++;
	}
} 
__global__ void parallel_processEdge(int *allvertex_devicein, int *alledge_devicein, 
	int *allweight_devicein, int z_device)
{
  int myId = threadIdx.x + blockDim.x * blockIdx.x;
	if (myId < edge_cnt) {
		if (allvertex_devicein[myId] == z_device)
		{ 
			printf("Thread %d looking for the Edge to be processed\n", threadIdx.x);
			int k_device = alledge_devicein[myId];
			int w_device = allweight_devicein[myId];
			printf("Edge {%d, %d, %d} found at myID:%d\n", z_device, k_device, w_device);

			if (!fixed[k_device]) {
				if (ifExistMWE(mwe, fromTo{z_device, k_device})) {
					fixed[k_device] = true;
							
					int t = atomicAdd(&T_index, 1);
					T[t] = fromTo{k_device, z_device}; // z is the parent of k
							
					int r = atomicAdd(&R_index, 1);
					R[r] = k_device;
					printf("Destination node is not fixed & also a minimum edge for Z:%d\n", z_device);
					printf("Adding k:%d to Tree & R for processing\n", k_device);
				}
				else if (dist[k_device] > w_device) {
					printf("Destination node is not fixed & NOT a minimum weight edge\n");
					printf("Adding k:%d to Q for inserting into Heap\n", k_device);
					dist[k_device] = w_device;
					parent[k_device] = z_device;

					if (!ifExist(Q, k_device)) {
						int q = atomicAdd(&Q_index, 1);
						Q[q] = k_device;
					}
				}
			}
			__syncthreads();     
		}
	}
}
void kernel_setup(Graph const &graph, int z_device){
	
	int threads = 512;
	int blocks = ceil(float(edge_cnt) / float(threads));

	const int ARRAY_BYTES = ARRAY_SIZE * ARRAY_SIZE * sizeof(int);

    int * allvertex_devicein, * alledge_devicein, * allweight_devicein;

    cudaMalloc((void **) &allvertex_devicein, ARRAY_BYTES);
	cudaMalloc((void **) &alledge_devicein, ARRAY_BYTES);
	cudaMalloc((void **) &allweight_devicein, ARRAY_BYTES);
	cudaMemcpy(allvertex_devicein, allvertex_in, ARRAY_BYTES, cudaMemcpyHostToDevice);
	gpuErrchk( cudaMemcpy(alledge_devicein, alledge_in, ARRAY_BYTES, cudaMemcpyHostToDevice) );
	gpuErrchk( cudaMemcpy(allweight_devicein, allweight_in, ARRAY_BYTES, cudaMemcpyHostToDevice) );
	parallel_processEdge<<<blocks, threads>>>
		(allvertex_devicein, alledge_devicein, allweight_devicein, z_device);

	gpuErrchk( cudaPeekAtLastError() );
  	gpuErrchk( cudaDeviceSynchronize() );
	cudaFree(allvertex_devicein);
	cudaFree(alledge_devicein);
	cudaFree(allweight_devicein);
};
void printMST(std::set<fromTo> T) 
{ 
	std::set<fromTo>::iterator it;
	for (it=T.begin(); it!=T.end(); it++) {
		fromTo e = *it; 
		printf("%d - %d\n", e.from, e.to); 
  }
} 
fromTo* primMST(Graph const &graph, int N, int source) 
{ 
	std::set<int>::iterator it;
	for(int i = 0; i < N; i ++) {
		parent[i] = -1;
		dist[i] = INT_MAX;
		fixed[i] = false;
	}
	dist[source] = 0; 
	H.push(distNode{source, dist[0]});

	initMWE(graph);

	for (int i = 0; i < graph.adjList.size(); i++) { 
		distNode d = H.top();
		H.pop();
		int j = d.node;
		printf("Popped minimum distance node:%d\n", j);
		if (!fixed[j]) {
			printf("Popped node is not fixed adding it to R\n");
			R[R_index] = j;
			R_index++;
			fixed[j] = true;
			if (parent[j] != -1) {
				T[T_index] = fromTo{j, parent[j]};
				T_index++;
			}
			while (R_index != 0){
				z_device = R[0];
				deleteElement(R, 0, ARRAY_SIZE);
				R_index--;
				printf("Calling kernel for processing edges of elements in R in parallel\n");
				kernel_setup(graph, z_device);
			}	
			
			while (Q_index != 0) {
					printf("Adding all elements from Q to Heap H\n");
					int z = Q[0];
					deleteElement(Q, 0, ARRAY_SIZE);
					Q_index--;
					if (!fixed[z]) {
						H.push(distNode{z, dist[z]});
					}
			}
		}
	}
	if (T_index == graph.adjList.size() -1) {
		return T;
	} else 
		return new fromTo[ARRAY_SIZE];
} 
int main() 
{ 
    std::vector<edge> edges;
    edges.push_back(edge{0, 1, 866});
    edges.push_back(edge{0, 2, 187});
    edges.push_back(edge{0, 3, 399});
    edges.push_back(edge{1, 5, 605});
    edges.push_back(edge{1, 10, 1720});
    edges.push_back(edge{1, 11, 888});
    edges.push_back(edge{1, 12, 409});
    edges.push_back(edge{2, 1, 739});
    edges.push_back(edge{2, 3, 213});
    edges.push_back(edge{2, 4, 541});
    edges.push_back(edge{2, 5, 759});
    edges.push_back(edge{2, 6, 1416});
    edges.push_back(edge{2, 7, 1391});
    edges.push_back(edge{2, 8, 2474});
    edges.push_back(edge{2, 9, 2586});
    edges.push_back(edge{2, 10, 2421});
    edges.push_back(edge{2, 11, 1625});
    edges.push_back(edge{2, 12, 765});
    edges.push_back(edge{3, 4, 330});
    edges.push_back(edge{3, 5, 547});
    edges.push_back(edge{3, 12, 561});
    edges.push_back(edge{4, 5, 226});
    edges.push_back(edge{4, 6, 912});
    edges.push_back(edge{5, 6, 689});
    edges.push_back(edge{5, 7, 731});
    edges.push_back(edge{5, 11, 1199});
    edges.push_back(edge{5, 12, 213});
    edges.push_back(edge{6, 7, 224});
    edges.push_back(edge{6, 8, 1378});
    edges.push_back(edge{7, 8, 1234});
    edges.push_back(edge{7, 11, 641});
    edges.push_back(edge{7, 12, 631});
    edges.push_back(edge{8, 9, 337});
    edges.push_back(edge{8, 11, 861});
    edges.push_back(edge{9, 10, 678});
    edges.push_back(edge{9, 11, 967});
    edges.push_back(edge{10, 11, 1024});
	  edges.push_back(edge{11, 12, 1013});
    Graph graph(edges, ARRAY_SIZE);
	  load_kernelArrays(graph);
    printGraph(graph);
	  int source;
	  for(int i = 0; i<nonEmptyIndices.size(); i++) {
			  if (nonEmptyIndices[i]) {
				  source = i;
			  break;
		    }
	  }
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);
	  primMST(graph, ARRAY_SIZE, source);
	  cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);
    printf("Parallel Elpased Time in ms:%f\n", elapsedTime);
    printf("\n====================================\n");
	  printf("Minimum Spanning Tree using Prim \n");
	  for (int i =0; i<T_index; i++) {
		  fromTo e = T[i]; 
		  printf("%d - %d\n", e.from, e.to); 
	  }
	return 0; 
}  

Input Graph
( 0, 1, 866 )( 0, 2, 187 )( 0, 3, 399 )( 1, 0, 866 )( 1, 5, 605 )( 1, 10, 1720 )( 1, 11, 888 )( 1, 12, 409 )( 1, 2, 739 )( 2, 0, 187 )( 2, 1, 739 )( 2, 3, 213 )( 2, 4, 541 )( 2, 5, 759 )( 2, 6, 1416 )( 2, 7, 1391 )( 2, 8, 2474 )( 2, 9, 2586 )( 2, 10, 2421 )( 2, 11, 1625 )( 2, 12, 765 )( 3, 0, 399 )( 3, 2, 213 )( 3, 4, 330 )( 3, 5, 547 )( 3, 12, 561 )( 4, 2, 541 )( 4, 3, 330 )( 4, 5, 226 )( 4, 6, 912 )( 5, 1, 605 )( 5, 2, 759 )( 5, 3, 547 )( 5, 4, 226 )( 5, 6, 689 )( 5, 7, 731 )( 5, 11, 1199 )( 5, 12, 213 )( 6, 2, 1416 )( 6, 4, 912 )( 6, 5, 689 )( 6, 7, 224 )( 6, 8, 1378 )( 7, 2, 1391 )( 7, 5, 731 )( 7, 6, 224 )( 7, 8, 1234 )( 7, 11, 641 )( 7, 12, 631 )( 8, 2, 2474 )( 8, 6, 1378 )( 8, 7, 1234 )( 8, 9, 337 )( 8, 11, 861 )( 9, 2, 2586 )( 9, 8, 337 )( 9, 10, 678 )( 9, 11, 967 )( 10, 1, 1720 )( 10, 2, 2421 )( 10, 9, 678 )( 10, 11, 1024 )( 11, 1, 888 )( 11, 2, 1625 )( 11, 5, 1199 )( 11, 7, 641 )( 11, 8, 861 )( 11, 9, 967 )( 11, 10, 1024 )( 11, 12, 1013 )( 12, 1, 409 )( 12, 2, 765

# GPU error check in Bitonic sort

In [ ]:
# GPU error check in Bitonic sort

%%cu
#include<iostream>
#include<fstream>
#include<cuda.h>
using namespace std;

#define NUM 1024

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

__device__ void swap(int &a, int &b){
    a = a+b;
    b = a-b;
    a = a-b;
}

__global__ static void bitonicSort(int * values)
{  extern __shared__ int shared[];
   const unsigned int tid = threadIdx.x;
   shared[tid] = values[tid];
   __syncthreads();
   for (unsigned int k = 2; k <= NUM; k *= 2){
     for (unsigned int j = k / 2; j>0; j /= 2){
        unsigned int ixj = tid ^ j;
        if (ixj > tid){
          if ((tid & k) == 0){
            if (shared[tid] > shared[ixj])
              swap(shared[tid], shared[ixj]); 
          }
          else if (shared[tid] < shared[ixj])
              swap(shared[tid], shared[ixj]);
          
        }
        __syncthreads();
     }
   } 
   values[tid] = shared[tid];
}


int main(int argc, char** argv)
{ 
 ifstream fin;
 fin.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::in);

 int values[NUM];
 for(int i = 0; i < NUM; i++)
    fin>>values[i];
 
 cout<<"Initial Array:"<<endl;
 for(int i = 0; i < NUM; i++)
    cout<<values[i]<<", ";
 cout<<endl;
 int * dvalues;

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
 
  cudaEventRecord(start);

 gpuErrchk(cudaMalloc((void**)&dvalues, sizeof(int) * NUM));
 gpuErrchk(cudaMemcpy(dvalues, values, sizeof(int) * NUM, cudaMemcpyHostToDevice));
 bitonicSort<<<1, NUM, sizeof(int) * NUM>>>(dvalues);
 gpuErrchk(cudaGetLastError());
 gpuErrchk(cudaMemcpy(values, dvalues, sizeof(int) * NUM, cudaMemcpyDeviceToHost));
 gpuErrchk(cudaFree(dvalues));

 cudaEventRecord(stop);
 cudaEventSynchronize(stop);
 fin.close();

 cout<<"Sorted Array:"<<endl;
 for(int i = 0; i < NUM; i++)
    cout<<values[i]<<", ";
 cout<<endl;


  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  cout<<"\nTime elpased in ms = "<<milliseconds;
 
 return 0;
}

Initial Array:
311, 525, 326, 198, 288, 53, 545, 147, 441, 164, 155, 111, 46, 8, 349, 360, 287, 411, 159, 1, 462, 65, 361, 542, 576, 430, 493, 287, 164, 39, 489, 63, 241, 77, 483, 146, 111, 353, 139, 172, 379, 497, 369, 291, 572, 477, 311, 1, 501, 218, 200, 321, 358, 477, 335, 270, 116, 375, 505, 443, 29, 77, 12, 490, 71, 141, 567, 512, 411, 120, 34, 494, 343, 50, 298, 524, 497, 420, 180, 506, 55, 122, 302, 243, 197, 400, 436, 166, 242, 520, 464, 96, 378, 377, 151, 344, 577, 248, 594, 529, 200, 230, 333, 34, 113, 358, 177, 105, 565, 136, 588, 349, 100, 149, 171, 40, 178, 2, 286, 244, 306, 308, 401, 485, 432, 11, 336, 351, 350, 520, 312, 585, 486, 44, 261, 258, 215, 68, 356, 76, 118, 1, 57, 135, 410, 13, 3, 142, 360, 395, 544, 283, 394, 21, 30, 540, 379, 582, 238, 276, 209, 464, 435, 116, 575, 559, 171, 3, 267, 225, 245, 217, 173, 431, 414, 178, 87, 292, 455, 360, 244, 265, 180, 277, 248, 347, 253, 93, 206, 583, 588, 314, 213, 357, 491, 391, 453, 176, 356, 30, 260, 226, 530, 309, 144, 4

#Device info

In [ ]:
# Device info

%%cu
#include <stdio.h> 

int main() {
  int nDevices;

  cudaGetDeviceCount(&nDevices);
  for (int i = 0; i < nDevices; i++) {
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, i);
    printf("Device Number: %d\n", i);
    printf("  Device name: %s\n", prop.name);
    printf("  Memory Clock Rate (KHz): %d\n",
           prop.memoryClockRate);
    printf("  Memory Bus Width (bits): %d\n",
           prop.memoryBusWidth);
    printf("  Peak Memory Bandwidth (GB/s): %f\n\n",
           2.0*prop.memoryClockRate*(prop.memoryBusWidth/8)/1.0e6);
  }
}

Device Number: 0
  Device name: Tesla T4
  Memory Clock Rate (KHz): 5001000
  Memory Bus Width (bits): 256
  Peak Memory Bandwidth (GB/s): 320.064000




#Device Properties

In [ ]:
# Device Properties

%%cu
#include <stdio.h>
 
// Print device properties
void printDevProp(cudaDeviceProp devProp)
{
    printf("Major revision number:         %d\n",  devProp.major);
    printf("Minor revision number:         %d\n",  devProp.minor);
    printf("Name:                          %s\n",  devProp.name);
    printf("Total global memory:           %u\n",  devProp.totalGlobalMem);
    printf("Total shared memory per block: %u\n",  devProp.sharedMemPerBlock);
    printf("Total registers per block:     %d\n",  devProp.regsPerBlock);
    printf("Warp size:                     %d\n",  devProp.warpSize);
    printf("Maximum memory pitch:          %u\n",  devProp.memPitch);
    printf("Maximum threads per block:     %d\n",  devProp.maxThreadsPerBlock);
    for (int i = 0; i < 3; ++i)
    printf("Maximum dimension %d of block:  %d\n", i, devProp.maxThreadsDim[i]);
    for (int i = 0; i < 3; ++i)
    printf("Maximum dimension %d of grid:   %d\n", i, devProp.maxGridSize[i]);
    printf("Clock rate:                    %d\n",  devProp.clockRate);
    printf("Total constant memory:         %u\n",  devProp.totalConstMem);
    printf("Texture alignment:             %u\n",  devProp.textureAlignment);
    printf("Concurrent copy and execution: %s\n",  (devProp.deviceOverlap ? "Yes" : "No"));
    printf("Number of multiprocessors:     %d\n",  devProp.multiProcessorCount);
    printf("Kernel execution timeout:      %s\n",  (devProp.kernelExecTimeoutEnabled ? "Yes" : "No"));
    return;
}
 
int main()
{
    // Number of CUDA devices
    int devCount;
    cudaGetDeviceCount(&devCount);
    printf("CUDA Device Query...\n");
    printf("There are %d CUDA devices.\n", devCount);
 
    // Iterate through devices
    for (int i = 0; i < devCount; ++i)
    {
        // Get device properties
        printf("\nCUDA Device #%d\n", i);
        cudaDeviceProp devProp;
        cudaGetDeviceProperties(&devProp, i);
        printDevProp(devProp);
    }
 
    printf("\nPress any key to exit...");
    char c;
    scanf("%c", &c);
 
    return 0;
}

CUDA Device Query...
There are 1 CUDA devices.

CUDA Device #0
Major revision number:         7
Minor revision number:         5
Name:                          Tesla T4
Total global memory:           2958819328
Total shared memory per block: 49152
Total registers per block:     65536
Warp size:                     32
Maximum memory pitch:          2147483647
Maximum threads per block:     1024
Maximum dimension 0 of block:  1024
Maximum dimension 1 of block:  1024
Maximum dimension 2 of block:  64
Maximum dimension 0 of grid:   2147483647
Maximum dimension 1 of grid:   65535
Maximum dimension 2 of grid:   65535
Clock rate:                    1590000
Total constant memory:         65536
Texture alignment:             512
Concurrent copy and execution: Yes
Number of multiprocessors:     40
Kernel execution timeout:      No

Press any key to exit...


#Matrix Addition using shared memory tiling

In [ ]:
# Matrix Addition using shared memory tiling

%%cu
#include<stdio.h>
#include<iostream>
#include<fstream>
#include<cuda.h>
using namespace std;

// Pull out matrix and shared memory tile size 
const int N = 1 << 6;
const int SHMEM_SIZE = 1 << 10;

#define row1 N
#define col1 N 

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}


__global__ void matrixAdd(const int *a, const int *b, int *c) {
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  __shared__ int s_a[SHMEM_SIZE];
  __shared__ int s_b[SHMEM_SIZE];

  // Sweep tile across matrix
  for (int i = 0; i < N; i += blockDim.x) {
    // Load in elements for this tile
    s_a[threadIdx.y * blockDim.x + threadIdx.x] = a[row * N + i + threadIdx.x];
    s_b[threadIdx.y * blockDim.x + threadIdx.x] = b[row * N + i + threadIdx.x];
    __syncthreads();
    c[row * N + i + threadIdx.x] = s_a[threadIdx.y * blockDim.x + threadIdx.x] + s_b[threadIdx.y * blockDim.x + threadIdx.x];
    __syncthreads();
  }

  //c[row * N + col] = s_a[threadIdx.y * blockDim.x + threadIdx.x] + s_b[threadIdx.y * blockDim.x + threadIdx.x];
}

int main()
{
 
  ifstream fin;
  fin.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::in);

  int a[row1][col1];
  int b[row1][col1];
  int c[row1][col1];
  int seq[row1][col1];
  int *d,*e,*f;
  int i,j,k;

  printf("\n Entering elements of first matrix\n");
  for(i=0;i<row1;i++)
  {
      for(j=0;j<col1;j++)
          {
              fin>>a[i][j];
              printf("%d\t",a[i][j]);
          }
    printf("\n");
  }
  printf("\n Enter elements of second matrix\n");
  for(i=0;i<row1;i++)
  {
      for(j=0;j<col1;j++)
          {
              fin>>b[i][j];
            printf("%d\t",b[i][j]);
          }
    printf("\n");
  }


  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaEventRecord(start);

  gpuErrchk(cudaMalloc((void **)&d,row1*col1*sizeof(int)));
  gpuErrchk(cudaMalloc((void **)&e,row1*col1*sizeof(int)));
  gpuErrchk(cudaMalloc((void **)&f,row1*col1*sizeof(int)));

  gpuErrchk(cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice));
  gpuErrchk(cudaMemcpy(e,b,row1*col1*sizeof(int),cudaMemcpyHostToDevice));

  int THREADS = 32;
  int BLOCKS = N / THREADS;

  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);

  matrixAdd<<<blocks, threads>>>(d, e, f);
  gpuErrchk(cudaGetLastError());

  gpuErrchk(cudaMemcpy(c,f,row1*col1*sizeof(int),cudaMemcpyDeviceToHost));
  gpuErrchk(cudaFree(d));
  gpuErrchk(cudaFree(e));
  gpuErrchk(cudaFree(f));

  fin.close();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  cout<<"\nTime elpased in ms = "<<milliseconds;
  printf("\n Product of two matrices using parallel execution:\n");
  for(i=0;i<row1;i++)
  {
      for(j=0;j<col1;j++)
      {
            printf("%d\t",c[i][j]);
      }
      printf("\n");
  }


  cudaEventRecord(start);
  for(i=0;i<row1;i++){
      for(j=0;j<col1;j++){
          seq[i][j] = a[i][j] + b[i][j];
      }
  }

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  cout<<"\nTime elpased in ms = "<<milliseconds;

  printf("\n Addition of two matrices using sequential execution:\n");
  for(i=0;i<row1;i++)
  {
      for(j=0;j<col1;j++)
      {
            printf("%d\t",seq[i][j]);
      }
      printf("\n");
  }

  printf("\n Error matrix:\n");
  for(i=0;i<row1;i++)
  {
      for(j=0;j<col1;j++)
      {
            printf("%d\t",seq[i][j] - c[i][j]);
      }
      printf("\n");
  }
  return 0;
}


 Entering elements of first matrix
311	525	326	198	288	53	545	147	441	164	155	111	46	8	349	360	287	411	159	1	462	65	361	542	576	430	493	287	164	39	489	63	241	77	483	146	111	353	139	172	379	497	369	291	572	477	311	1	501	218	200	321	358	477	335	270	116	375	505	443	29	77	12	490	
71	141	567	512	411	120	34	494	343	50	298	524	497	420	180	506	55	122	302	243	197	400	436	166	242	520	464	96	378	377	151	344	577	248	594	529	200	230	333	34	113	358	177	105	565	136	588	349	100	149	171	40	178	2	286	244	306	308	401	485	432	11	336	351	
350	520	312	585	486	44	261	258	215	68	356	76	118	1	57	135	410	13	3	142	360	395	544	283	394	21	30	540	379	582	238	276	209	464	435	116	575	559	171	3	267	225	245	217	173	431	414	178	87	292	455	360	244	265	180	277	248	347	253	93	206	583	588	314	
213	357	491	391	453	176	356	30	260	226	530	309	144	455	400	356	120	108	505	206	286	588	521	288	349	405	237	314	191	475	589	338	38	157	516	400	242	17	109	305	233	261	192	147	432	412	185	4	47	344	382	269	528	180	226	398	215	107	472	28	

#Matrix Multiplication using shared memory tiling

In [ ]:
# Matrix Multiplication using shared memory tiling

%%cu
#include<stdio.h>
#include<iostream>
#include<fstream>
#include<cuda.h>
using namespace std;

// Pull out matrix and shared memory tile size 
const int N = 1 << 6;
const int SHMEM_SIZE = 1 << 10;

#define row1 N
#define col1 N 
#define row2 N
#define col2 N

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}


__global__ void matrixMul(const int *a, const int *b, int *c) {
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  __shared__ int s_a[SHMEM_SIZE];
  __shared__ int s_b[SHMEM_SIZE];

  int tmp = 0;

  // Sweep tile across matrix
  for (int i = 0; i < N; i += blockDim.x) {
    // Load in elements for this tile
    s_a[threadIdx.y * blockDim.x + threadIdx.x] = a[row * N + i + threadIdx.x];
    s_b[threadIdx.y * blockDim.x + threadIdx.x] = b[i * N + threadIdx.y * N + col];
    __syncthreads();

    // Do matrix multiplication on the small matrix
    for (int j = 0; j < blockDim.x; j++) {
      tmp += s_a[threadIdx.y * blockDim.x + j] * s_b[j * blockDim.x + threadIdx.x];
    }
    __syncthreads();
  }

  c[row * N + col] = tmp;
}

int main()
{
 
    ifstream fin;
    fin.open("/content/drive/MyDrive/GPU Lab/randomNumbers.txt", ios::in);
 
    int a[row1][col1];
    int b[row2][col2];
    int c[row1][col2];
    int seq[row1][col2];
    int *d,*e,*f;
    int i,j,k;

    printf("\n Entering elements of first matrix\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col1;j++)
            {
                fin>>a[i][j];
                printf("%d\t",a[i][j]);
            }
     printf("\n");
    }
    printf("\n Enter elements of second matrix\n");
        for(i=0;i<row2;i++)
        {
            for(j=0;j<col2;j++)
                {
                    fin>>b[i][j];
                 printf("%d\t",b[i][j]);
                }
         printf("\n");
        }


  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
 
  cudaEventRecord(start);

    gpuErrchk(cudaMalloc((void **)&d,row1*col1*sizeof(int)));
    gpuErrchk(cudaMalloc((void **)&e,row2*col2*sizeof(int)));
    gpuErrchk(cudaMalloc((void **)&f,row1*col2*sizeof(int)));

 gpuErrchk(cudaMemcpy(d,a,row1*col1*sizeof(int),cudaMemcpyHostToDevice));
 gpuErrchk(cudaMemcpy(e,b,row2*col2*sizeof(int),cudaMemcpyHostToDevice));

  int THREADS = 32;
  int BLOCKS = N / THREADS;

  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);

  matrixMul<<<blocks, threads>>>(d, e, f);
  gpuErrchk(cudaGetLastError());

 gpuErrchk(cudaMemcpy(c,f,row1*col2*sizeof(int),cudaMemcpyDeviceToHost));
 gpuErrchk(cudaFree(d));
 gpuErrchk(cudaFree(e));
 gpuErrchk(cudaFree(f));
  
  fin.close();
 
 cudaEventRecord(stop);
 cudaEventSynchronize(stop);
 
 float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  cout<<"\nTime elpased in ms = "<<milliseconds;
 printf("\n Product of two matrices using parallel execution:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",c[i][j]);
        }
        printf("\n");
    }
    
    
    cudaEventRecord(start);
    for(i=0;i<row1;i++){
        for(j=0;j<col2;j++){
            seq[i][j] = 0;
            for(k=0;k<col1;k++){
                seq[i][j] += a[i][k] * b[k][j];
            }
        }
    }
 
 cudaEventRecord(stop);
 cudaEventSynchronize(stop);
 
 milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  cout<<"\nTime elpased in ms = "<<milliseconds;
 
    printf("\n Product of two matrices using sequential execution:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",seq[i][j]);
        }
        printf("\n");
    }

    printf("\n Error matrix:\n");
    for(i=0;i<row1;i++)
    {
        for(j=0;j<col2;j++)
        {
              printf("%d\t",seq[i][j] - c[i][j]);
        }
        printf("\n");
    }


    
    return 0;
}


 Entering elements of first matrix
311	525	326	198	288	53	545	147	441	164	155	111	46	8	349	360	287	411	159	1	462	65	361	542	576	430	493	287	164	39	489	63	241	77	483	146	111	353	139	172	379	497	369	291	572	477	311	1	501	218	200	321	358	477	335	270	116	375	505	443	29	77	12	490	
71	141	567	512	411	120	34	494	343	50	298	524	497	420	180	506	55	122	302	243	197	400	436	166	242	520	464	96	378	377	151	344	577	248	594	529	200	230	333	34	113	358	177	105	565	136	588	349	100	149	171	40	178	2	286	244	306	308	401	485	432	11	336	351	
350	520	312	585	486	44	261	258	215	68	356	76	118	1	57	135	410	13	3	142	360	395	544	283	394	21	30	540	379	582	238	276	209	464	435	116	575	559	171	3	267	225	245	217	173	431	414	178	87	292	455	360	244	265	180	277	248	347	253	93	206	583	588	314	
213	357	491	391	453	176	356	30	260	226	530	309	144	455	400	356	120	108	505	206	286	588	521	288	349	405	237	314	191	475	589	338	38	157	516	400	242	17	109	305	233	261	192	147	432	412	185	4	47	344	382	269	528	180	226	398	215	107	472	28	

#openACC

In [ ]:
!git clone https://github.com/ENCCS/OpenACC-CUDA-beginners.git

Cloning into 'OpenACC-CUDA-beginners'...
remote: Enumerating objects: 1380, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 1380 (delta 97), reused 127 (delta 76), pack-reused 1189
Receiving objects: 100% (1380/1380), 14.14 MiB | 25.71 MiB/s, done.
Resolving deltas: 100% (651/651), done.


In [ ]:
!ls

drive  OpenACC-CUDA-beginners  sample_data


In [ ]:
%cd OpenACC-CUDA-beginners
!git checkout colab_gcc

/content/OpenACC-CUDA-beginners
Branch 'colab_gcc' set up to track remote branch 'colab_gcc' from 'origin'.
Switched to a new branch 'colab_gcc'


In [ ]:
!git branch

* colab_gcc
  main


In [ ]:
%cd /content/OpenACC-CUDA-beginners/examples/OpenACC/vector-sum/solution/c

/content/OpenACC-CUDA-beginners/examples/OpenACC/vector-sum/solution/c


In [ ]:
!ls

Makefile  sum.c  sum_kernels.c	sum_parallel.c	sum_parallel_call_function.c


In [ ]:
!gcc -fopenacc -o matrixMultiply_c matrixMultiply.c

matrixMultiply.c: In function ‘main’:
matrixMultiply.c:39:18: error: ‘new’ undeclared (first use in this function)
  int** matrixA = new int* [a1];
                  ^~~
matrixMultiply.c:39:18: note: each undeclared identifier is reported only once for each function it appears in
matrixMultiply.c:39:22: error: expected ‘,’ or ‘;’ before ‘int’
  int** matrixA = new int* [a1];
                      ^~~
matrixMultiply.c:40:22: error: expected ‘,’ or ‘;’ before ‘int’
  int** matrixB = new int* [a2];
                      ^~~
matrixMultiply.c:41:21: error: expected ‘,’ or ‘;’ before ‘int’
  int** Answer = new int* [a1];
                     ^~~


In [ ]:
!./matrixMultiply_c

Reduction sum: 1.2020569031119110


#OpenCL

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
 
#ifdef __APPLE__
#include <OpenCL/opencl.h>
#else
#include <CL/cl.h>
#endif
 
#define MAX_SOURCE_SIZE (0x100000)
 
int main(void) {
    // Create the two input vectors
    int i;
    const int LIST_SIZE = 1024;
    int *A = (int*)malloc(sizeof(int)*LIST_SIZE);
    int *B = (int*)malloc(sizeof(int)*LIST_SIZE);
    for(i = 0; i < LIST_SIZE; i++) {
        A[i] = i;
        B[i] = LIST_SIZE - i;
    }
 
    // Load the kernel source code into the array source_str
    FILE *fp;
    char *source_str;
    size_t source_size;
 
    fp = fopen("vector_add_kernel.cl", "r");
    if (!fp) {
        fprintf(stderr, "Failed to load kernel.\n");
        exit(1);
    }
    source_str = (char*)malloc(MAX_SOURCE_SIZE);
    source_size = fread( source_str, 1, MAX_SOURCE_SIZE, fp);
    fclose( fp );
 
    // Get platform and device information
    cl_platform_id platform_id = NULL;
    cl_device_id device_id = NULL;   
    cl_uint ret_num_devices;
    cl_uint ret_num_platforms;
    cl_int ret = clGetPlatformIDs(1, &platform_id, &ret_num_platforms);
    ret = clGetDeviceIDs( platform_id, CL_DEVICE_TYPE_DEFAULT, 1, 
            &device_id, &ret_num_devices);
 
    // Create an OpenCL context
    cl_context context = clCreateContext( NULL, 1, &device_id, NULL, NULL, &ret);
 
    // Create a command queue
    cl_command_queue command_queue = clCreateCommandQueue(context, device_id, 0, &ret);
 
    // Create memory buffers on the device for each vector 
    cl_mem a_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY, 
            LIST_SIZE * sizeof(int), NULL, &ret);
    cl_mem b_mem_obj = clCreateBuffer(context, CL_MEM_READ_ONLY,
            LIST_SIZE * sizeof(int), NULL, &ret);
    cl_mem c_mem_obj = clCreateBuffer(context, CL_MEM_WRITE_ONLY, 
            LIST_SIZE * sizeof(int), NULL, &ret);
 
    // Copy the lists A and B to their respective memory buffers
    ret = clEnqueueWriteBuffer(command_queue, a_mem_obj, CL_TRUE, 0,
            LIST_SIZE * sizeof(int), A, 0, NULL, NULL);
    ret = clEnqueueWriteBuffer(command_queue, b_mem_obj, CL_TRUE, 0, 
            LIST_SIZE * sizeof(int), B, 0, NULL, NULL);
 
    // Create a program from the kernel source
    cl_program program = clCreateProgramWithSource(context, 1, 
            (const char **)&source_str, (const size_t *)&source_size, &ret);
 
    // Build the program
    ret = clBuildProgram(program, 1, &device_id, NULL, NULL, NULL);
 
    // Create the OpenCL kernel
    cl_kernel kernel = clCreateKernel(program, "vector_add", &ret);
 
    // Set the arguments of the kernel
    ret = clSetKernelArg(kernel, 0, sizeof(cl_mem), (void *)&a_mem_obj);
    ret = clSetKernelArg(kernel, 1, sizeof(cl_mem), (void *)&b_mem_obj);
    ret = clSetKernelArg(kernel, 2, sizeof(cl_mem), (void *)&c_mem_obj);
 
    // Execute the OpenCL kernel on the list
    size_t global_item_size = LIST_SIZE; // Process the entire lists
    size_t local_item_size = 64; // Divide work items into groups of 64
    ret = clEnqueueNDRangeKernel(command_queue, kernel, 1, NULL, 
            &global_item_size, &local_item_size, 0, NULL, NULL);
 
    // Read the memory buffer C on the device to the local variable C
    int *C = (int*)malloc(sizeof(int)*LIST_SIZE);
    ret = clEnqueueReadBuffer(command_queue, c_mem_obj, CL_TRUE, 0, 
            LIST_SIZE * sizeof(int), C, 0, NULL, NULL);
 
    // Display the result to the screen
    for(i = 0; i < LIST_SIZE; i++)
        printf("%d + %d = %d\n", A[i], B[i], C[i]);
 
    // Clean up
    ret = clFlush(command_queue);
    ret = clFinish(command_queue);
    ret = clReleaseKernel(kernel);
    ret = clReleaseProgram(program);
    ret = clReleaseMemObject(a_mem_obj);
    ret = clReleaseMemObject(b_mem_obj);
    ret = clReleaseMemObject(c_mem_obj);
    ret = clReleaseCommandQueue(command_queue);
    ret = clReleaseContext(context);
    free(A);
    free(B);
    free(C);
    return 0;
}

/tmp/tmpxft_000001fb_00000000-11_4e85f181-283f-499e-b8af-c57e47e3cf4d.o: In function `main':
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x168): undefined reference to `clGetPlatformIDs'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x195): undefined reference to `clGetDeviceIDs'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x1c6): undefined reference to `clCreateContext'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x1e9): undefined reference to `clCreateCommandQueue'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x212): undefined reference to `clCreateBuffer'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x23b): undefined reference to `clCreateBuffer'
tmpxft_000001fb_00000000-6_4e85f181-283f-499e-b8af-c57e47e3cf4d.cudafe1.cpp:(.text+0x264): undefined reference t

# ***Lab eval questions***

## openACC program for Prim's MST 

In [ ]:
// openACC program for Prim's MST and maximum value in the file
// 9.txt

#include <limits.h>
#include <stdbool.h>
#include <stdio.h>
#ifdef _OPENACC
#include <openacc.h>
#endif

int V = 5;

int findSQRT(int number)
{
    int start = 0, end = number;
    int mid;
    int ans;
    while (start <= end) {
        mid = (start + end) / 2;
        if (mid * mid == number) {
            ans = mid;
            break;
        }
 
        if (mid * mid < number) {
  
            ans=start;
            start = mid + 1;
        }

        else {
            end = mid - 1;
        }
    } 
    return ans;
}

int minKey(int key[], bool mstSet[])
{
    // Initialize min value
    int min = INT_MAX, min_index;
  
  
        for (int v = 0; v < V; v++){
            if (mstSet[v] == false && key[v] < min)
                min = key[v], min_index = v;
        }  
    
    return min_index;
}
  
int printMST(int parent[], int graph[V][V])
{
    printf("\tEdge\t\t\t\tWeight\n");
 #pragma acc parallel
    {
#pragma acc loop
        for (int i = 1; i < V; i++){
            printf("%d\t-\t%d\t\t\t%d \n", parent[i], i, graph[i][parent[i]]);
        }
     }
}
  
void primMST(int graph[V][V])
{
    int parent[V];
    int key[V];
    bool mstSet[V];
  #pragma acc parallel
    {
#pragma acc loop
    for (int i = 0; i < V; i++)
        key[i] = INT_MAX, mstSet[i] = false;
    }
  
    key[0] = 0;
    parent[0] = -1; // First node is always root of MST
  #pragma acc parallel
    {
#pragma acc loop
    for (int count = 0; count < V - 1; count++) {
        int u = minKey(key, mstSet);
  
        mstSet[u] = true;
  
        for (int v = 0; v < V; v++)
  
            if (graph[u][v] && mstSet[v] == false && graph[u][v] < key[v])
                parent[v] = u, key[v] = graph[u][v];
    }
    }
  
    printMST(parent, graph);
}
  
int main()
{
    int x;
    int count = 0;
    int *input;
    FILE * fp = fopen("/content/drive/MyDrive/GPU Lab/9.txt", "r"); 
    if (fp == NULL) return 1; 
    while (fscanf(fp, "%d", &x) == 1) {
      count++;
    }
    fclose(fp);
    printf("Count = %d\n", count);

    V = findSQRT(count);
    printf("Vertices = %d\n", V);
    int graph[V][V]; 
    FILE * fp1 = fopen("/content/drive/MyDrive/GPU Lab/9.txt", "r"); 
    if (fp1 == NULL) return 1; 
    for(int i=0;i<V;i++){
      for(int j=0;j<V;j++){
          fscanf(fp1, "%d", &x);
          graph[i][j] = x;
      }
    }
    printf("Adjacency Matrix:\n");
    for(int i=0;i<V;i++){
      for(int j=0;j<V;j++){
          printf("%d\t",graph[i][j]);
      }
      printf("\n");
    }

    fclose(fp1);
  
    // Print the solution
    primMST(graph);

    int max = 0;
    FILE * fp2 = fopen("/content/drive/MyDrive/GPU Lab/9.txt", "r"); 
    if (fp2 == NULL) return 1; 
          for(int i=0;i<count;i++){
                fscanf(fp2, "%d", &x);
                if(x>max){
                  max = x;
                }
          }
    fclose(fp2);
    printf("Maximum element in the File is: %d", max);
  
    return 0;
}

## matrix multiplication using tiling operations in openAcc

In [ ]:
/*The data from the files which will be provided to you. Using OpenAcc programing
shape the data into a matrix form. you need to perform matrix multiplication using
tiling operations on the data in these matrixes. Calculate the average values given
to you in these files. Calculation execution time for all the above calculations.
Data files 21.txt 32.txt*/

#ifdef _OPENACC
#include <openacc.h>
#endif

#include <iostream>
#include <cmath>
#include <chrono>
#include <fstream>

using namespace std::chrono;

int main()
{
    std::ifstream fp1, fp2;
    fp1.open("C:/Users/abhig/Downloads/21.txt", std::ios::in);
    fp2.open("C:/Users/abhig/Downloads/32.txt", std::ios::in);

    int t1 = 0, t2 = 0, total1 = 0, total2 = 0, x = 0;

    while (!fp1.eof()) {
        t1++;
        fp1 >> x;
        total1 += x;
    }
    x = 0;
    while (!fp2.eof()) {
        t2++;
        fp2 >> x;
        total2 += x;
    }

    int a1 = sqrt((double)t1);
    int b1 = t1 / a1;
    int a2 = b1;
    int b2 = t2 / a2;

    int i = 0, j = 0, k = 0;
    int** matrixA = new int* [a1];
    int** matrixB = new int* [a2];
    int** Answer = new int* [a1];

    for (i = 0; i < a1; ++i) {
        matrixA[i] = new int[b1];
    }
    for (i = 0; i < a2; ++i) {
        matrixB[i] = new int[b2];
    }
    for (i = 0; i < a1; ++i) {
        Answer[i] = new int[b2];
    }

    auto start = high_resolution_clock::now();
    
	#pragma acc parallel num_gangs 1024
    {
        #pragma acc loop gang
        for (int i = 0; i < a1; ++i)
        {
            #pragma acc loop
            for (int j = 0; j < b1; ++j)
            {
                fp1 >> matrixA[i][j];
            }
        }
    }

	#pragma acc parallel num_gangs 1024
    {
        #pragma acc loop gang
        for (int i = 0; i < a2; ++i)
        {
            #pragma acc loop
            for (int j = 0; j < b2; ++j)
            {
                fp2 >> matrixB[i][j];
            }
        }
    }

    #pragma acc parallel num_gangs 1024
    {
        #pragma acc loop gang
        for (int i = 0; i < a1; ++i) {
            #pragma acc loop 
            for (int j = 0; j < b2; ++j) {
                for (int k = 0; k < a2; ++k) {
                    Answer[i][j] += matrixA[i][k] * matrixB[k][j];
                }
            }
        }
    }

    float avg1 = (float)total1 / (float)t1;
    float avg2 = (float)total2 / (float)t2;
    
    std::cout << "Avg File 1:" << avg1 << "\nAvg File 2:" << avg2 << std::endl;

    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<nanoseconds>(stop - start);
    std::cout << "\nTime taken: " << (duration.count()/1000) << " Mili Seconds" << std::endl;

    fp1.close();
    fp2.close();
    return 0;
}

##File to array conversion

In [ ]:
// storing file to an array
fin.open("/content/drive/My Drive/Colab Notebooks/22.txt",std ::ios::in);
  while(!fin.eof()){
      fin>>line;
      a[i]=(stoi(line));
      i++;
      count++;
  }
  printf("count: %d\n",count);
  printf("Unsorted Array is:");
	for(int i=0;i<count;i++){
      printf("%d ",a[i]);
  }
  printf("\n");
   int n=sqrt(count);
  int k=0;
  printf("In matrix form:\n");
  for(int i=0;i<n;i++){
      for(int j=0;j<n;j++){
          printf("%d ",a[k++]);
      }
      printf("\n");
  }

##matrix multiplication using shared memory tiling CUDA

In [ ]:
# This program computes matrix multiplication using shared memory tiling CUDA
%%cu
#include <algorithm>
#include <cassert>
#include <cstdlib>
#include <functional>
#include <iostream>
#include <vector>

using std::cout;
using std::generate;
using std::vector;

// Pull out matrix and shared memory tile size 
const int N = 1 << 10;
const int SHMEM_SIZE = 1 << 10;

__global__ void matrixMul(const int *a, const int *b, int *c) {
  // Compute each thread's global row and column index
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  // Statically allocated shared memory
  __shared__ int s_a[SHMEM_SIZE];
  __shared__ int s_b[SHMEM_SIZE];

  // Accumulate in temporary variable
  int tmp = 0;

  // Sweep tile across matrix
  for (int i = 0; i < N; i += blockDim.x) {
    // Load in elements for this tile
    s_a[threadIdx.y * blockDim.x + threadIdx.x] = a[row * N + i + threadIdx.x];
    s_b[threadIdx.y * blockDim.x + threadIdx.x] =
        b[i * N + threadIdx.y * N + col];

    // Wait for both tiles to be loaded in before doing computation
    __syncthreads();

    // Do matrix multiplication on the small matrix
    for (int j = 0; j < blockDim.x; j++) {
      tmp +=
          s_a[threadIdx.y * blockDim.x + j] * s_b[j * blockDim.x + threadIdx.x];
    }

    // Wait for all threads to finish using current tiles before loading in new
    // ones
    __syncthreads();
  }

  // Write back results
  c[row * N + col] = tmp;
}

// Check result on the CPU
void verify_result(vector<int> &a, vector<int> &b, vector<int> &c) {
  // For every row...
  for (int i = 0; i < N; i++) {
    // For every column...
    for (int j = 0; j < N; j++) {
      // For every element in the row-column pair
      int tmp = 0;
      for (int k = 0; k < N; k++) {
        // Accumulate the partial results
        tmp += a[i * N + k] * b[k * N + j];
      }

      // Check against the CPU result
      assert(tmp == c[i * N + j]);
    }
  }
}

int main() {
  // Size (in bytes) of matrix
  size_t bytes = N * N * sizeof(int);

  // Host vectors
  vector<int> h_a(N * N);
  vector<int> h_b(N * N);
  vector<int> h_c(N * N);

  // Initialize matrices
  generate(h_a.begin(), h_a.end(), []() { return rand() % 100; });
  generate(h_b.begin(), h_b.end(), []() { return rand() % 100; });

  // Allocate device memory
  int *d_a, *d_b, *d_c;
  cudaMalloc(&d_a, bytes);
  cudaMalloc(&d_b, bytes);
  cudaMalloc(&d_c, bytes);

  // Copy data to the device
  cudaMemcpy(d_a, h_a.data(), bytes, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, h_b.data(), bytes, cudaMemcpyHostToDevice);

  // Threads per CTA dimension
  int THREADS = 32;

  // Blocks per grid dimension (assumes THREADS divides N evenly)
  int BLOCKS = N / THREADS;

  // Use dim3 structs for block  and grid dimensions
  dim3 threads(THREADS, THREADS);
  dim3 blocks(BLOCKS, BLOCKS);

  // Launch kernel
  matrixMul<<<blocks, threads>>>(d_a, d_b, d_c);

  // Copy back to the host
  cudaMemcpy(h_c.data(), d_c, bytes, cudaMemcpyDeviceToHost);

  // Check result
  verify_result(h_a, h_b, h_c);

  cout << "COMPLETED SUCCESSFULLY\n";

  // Free memory on device
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);

  return 0;
}

##odd even sort CUDA

In [ ]:
# odd even sort CUDA
%%cu
#include<bits/stdc++.h>
using namespace std;
_global_ void oddeven(int* x,int I,int n)
{
  int id=blockIdx.x;
	if(I==0 && ((id*2+1)< n)){ //0,1 2,3...
	  if(x[id*2]>x[id*2+1]){
		  int X=x[id*2];
		  x[id*2]=x[id*2+1];
		  x[id*2+1]=X;
  	}
  }
  
	if(I==1 && ((id*2+2)< n)){ //1,2 3,4...
    if(x[id*2+1]>x[id*2+2]){
			int X=x[id*2+1];
			x[id*2+1]=x[id*2+2];
			x[id*2+2]=X;
		}
	}
}
int main()
  {
  
  int *a,*b;
  int size=sizeof(int)*100000;
  a=(int *)malloc(size);
  b=(int *)malloc(size);
  fstream fin;
  string line;
  int i=0,count=0;
  fin.open("/content/drive/My Drive/Colab Notebooks/22.txt",std ::ios::in);
  while(!fin.eof()){
      fin>>line;
      a[i]=(stoi(line));
      i++;
      count++;
  }
  printf("count: %d\n",count);
  printf("Unsorted Array is:");
	for(int i=0;i<count;i++){
      printf("%d ",a[i]);
  }
  printf("\n");
   int n=sqrt(count);
  int k=0;
  printf("In matrix form:\n");
  for(int i=0;i<n;i++){
      for(int j=0;j<n;j++){
          printf("%d ",a[k++]);
      }
      printf("\n");
  }
	int *c;
 
	cudaMalloc((void **)&c, size);
	cudaMemcpy(c,a,size,cudaMemcpyHostToDevice);
	for(int i=0;i<count;i++){
		//int size=n/2;
		oddeven<<<count/2,1>>>(c,i%2,count);
	}
	cudaMemcpy(b,c,size,cudaMemcpyDeviceToHost);

  printf("Sorted Array is:\n");
  for(int i=0; i<count; i++){
    printf("%d ",b[i]);
	}
  k=0;
  printf("\nSorted In matrix form:\n");
  for(int i=0;i<n;i++){
      for(int j=0;j<n;j++){
          printf("%d ",b[k++]);
      }
      printf("\n");
  }
  printf("\n maximum value in the file is: %d",b[count-1]);
  printf("\n");

}

##bitonic sort cuda

In [ ]:
#bitonic sort cuda
%%cu
/*
 * Parallel bitonic sort using CUDA.
 * Compile with
 * nvcc -arch=sm_11 bitonic_sort.cu
 * Based on http://www.tools-of-computing.com/tc/CS/Sorts/bitonic_sort.htm
 * License: BSD 3
 */

#include <stdlib.h>
#include <stdio.h>
#include <time.h>

/* Every thread gets exactly one value in the unsorted array. */
#define THREADS 512 // 2^9
#define BLOCKS 32768 // 2^15
#define NUM_VALS THREADS*BLOCKS

void print_elapsed(clock_t start, clock_t stop)
{
  double elapsed = ((double) (stop - start)) / CLOCKS_PER_SEC;
  printf("Elapsed time: %.3fs\n", elapsed);
}

float random_float()
{
  return (float)rand()/(float)RAND_MAX;
}

void array_print(float *arr, int length) 
{
  int i;
  for (i = 0; i < length; ++i) {
    printf("%1.3f ",  arr[i]);
  }
  printf("\n");
}

void array_fill(float *arr, int length)
{
  srand(time(NULL));
  int i;
  for (i = 0; i < length; ++i) {
    arr[i] = random_float();
  }
}

__global__ void bitonic_sort_step(float *dev_values, int j, int k)
{
  unsigned int i, ixj; /* Sorting partners: i and ixj */
  i = threadIdx.x + blockDim.x * blockIdx.x;
  ixj = i^j;

  /* The threads with the lowest ids sort the array. */
  if ((ixj)>i) {
    if ((i&k)==0) {
      /* Sort ascending */
      if (dev_values[i]>dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
    if ((i&k)!=0) {
      /* Sort descending */
      if (dev_values[i]<dev_values[ixj]) {
        /* exchange(i,ixj); */
        float temp = dev_values[i];
        dev_values[i] = dev_values[ixj];
        dev_values[ixj] = temp;
      }
    }
  }
}

/**
 * Inplace bitonic sort using CUDA.
 */
void bitonic_sort(float *values)
{
  float *dev_values;
  size_t size = NUM_VALS * sizeof(float);

  cudaMalloc((void**) &dev_values, size);
  cudaMemcpy(dev_values, values, size, cudaMemcpyHostToDevice);

  dim3 blocks(BLOCKS,1);    /* Number of blocks   */
  dim3 threads(THREADS,1);  /* Number of threads  */

  int j, k;
  /* Major step */
  for (k = 2; k <= NUM_VALS; k <<= 1) {
    /* Minor step */
    for (j=k>>1; j>0; j=j>>1) {
      bitonic_sort_step<<<blocks, threads>>>(dev_values, j, k);
    }
  }
  cudaMemcpy(values, dev_values, size, cudaMemcpyDeviceToHost);
  cudaFree(dev_values);
}

int main(void)
{
  clock_t start, stop;

  float *values = (float*) malloc( NUM_VALS * sizeof(float));
  array_fill(values, NUM_VALS);

  start = clock();
  bitonic_sort(values); /* Inplace */
  stop = clock();

  print_elapsed(start, stop);
}

##matrix addition using cuda

In [ ]:
#matrix addition using cuda
%%cu
/*files used 61.txt and 62.txt */
#include <cuda_runtime.h>
#include <iostream>
#include <stdio.h>
#include <fstream>
#include <sys/time.h>
#define FILE_SIZE 1000
using namespace std;

#define N 23

_global_ void MatAdd(int A[][N], int B[][N], int C[][N]){
           int i = threadIdx.x;
           int j = threadIdx.y;

           C[i][j] = A[i][j] + B[i][j];
       }


int main(){

 int z[1000];
  ifstream is("61.txt");
  int cnt= 0;
  int x;
  while (cnt < 600 && is >> x)
    z[cnt++] = x;

  int a[N][N];
  
   for(int i=0;i<29;i++)
  {
      for(int j=0;j<29;j++)
      {
          a[i][j]=z[i*j+i+j];       
      }
  }    
  is.close();
  
  cout<<"The integers of 1st array are:"<<"\n";
  for(int i=0;i<29;i++)
  {
      for(int j=0;j<29;j++)
      {
          cout<<a[i][j]<<" ";       
      }
   cout<<endl;
  }
  cout<<"\n";

ifstream isx("62.txt");
int r[1000];
  int cont= 0;
  int x1;
  while (cont < 600 && isx >> x1)
    r[cont++] = x1;

int B[N][N] ;

for(int i=0;i<29;i++)
  {
      for(int j=0;j<29;j++)
      {
          B[i][j]=r[i*j+i+j];       
      }
  }

isx.close();
cout<<"The integers of 2nd array are:"<<"\n";
  for(int i=0;i<29;i++)
  {
      for(int j=0;j<29;j++)
      {
          cout<<B[i][j]<<" ";       
      }
   cout<<endl;
  }
  cout<<"\n";
int C[N][N];    

int (*pA)[N], (*pB)[N], (*pC)[N];

cudaMalloc((void**)&pA, (N*N)*sizeof(int));
cudaMalloc((void**)&pB, (N*N)*sizeof(int));
cudaMalloc((void**)&pC, (N*N)*sizeof(int));

cudaMemcpy(pA, a, (N*N)*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(pB, B, (N*N)*sizeof(int), cudaMemcpyHostToDevice);
cudaMemcpy(pC, C, (N*N)*sizeof(int), cudaMemcpyHostToDevice);

int numBlocks = 1;
dim3 threadsPerBlock(N,N);
MatAdd<<<numBlocks,threadsPerBlock>>>(pA,pB,pC);

cudaMemcpy(C, pC, (N*N)*sizeof(int), cudaMemcpyDeviceToHost);

int i, j; printf("C = \n");
for(i=0;i<N;i++){
    for(j=0;j<N;j++){
        printf("%d ", C[i][j]);
    }
    printf("\n");
}

cudaFree(pA); 
cudaFree(pB); 
cudaFree(pC);

printf("\n");

return 0;
}

#***Previous Sem Questions***

##multiple blocks 1 thread vector addition

In [ ]:
#multiple blocks 1 thread vector addition Assign3 Q3
%%cu
#define N 512
#include<iostream>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecAdd(int *a, int* b, int* c);

int main(){
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
        fin>>b[i];
    }

    int size = N*sizeof(int);
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size,cudaMemcpyHostToDevice);
    
    vecAdd<<<N,1>>>(dev_a,dev_b,dev_c);
    
    cudaMemcpy(c, dev_c, size,cudaMemcpyDeviceToHost);
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nb[]\n";
    for(int i=0;i<N;i++){
        cout<<b[i]<<" ";
    }
    cout<<"\nc[]\n";
    for(int i=0;i<N;i++){
        cout<<c[i]<<" ";
    }
    fin.close();
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecAdd (int *a, int *b, int *c) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i<N){
      c[i] = a[i] + b[i];
    }
}

##1 block multiple threads vector subtraction

In [ ]:
#1 block multiple threads vector subtraction Assign3 Q4
%%cu
#define N 512
#include<iostream>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecSub(int *a, int* b, int* c);

int main(){
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
        fin>>b[i];
    }

    int size = N*sizeof(int);
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size,cudaMemcpyHostToDevice);
    
    vecSub<<<1,N>>>(dev_a,dev_b,dev_c);
    
    cudaMemcpy(c, dev_c, size,cudaMemcpyDeviceToHost);
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nb[]\n";
    for(int i=0;i<N;i++){
        cout<<b[i]<<" ";
    }
    cout<<"\nc[]\n";
    for(int i=0;i<N;i++){
        cout<<c[i]<<" ";
    }
    fin.close();
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecSub (int *a, int *b, int *c) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i<N){
      c[i] = a[i] - b[i];
    }
}

## Vector Addition

In [ ]:
#multiple block multiple thread vector addition
%%cu
#define T 1024
#define N 10000
#include<iostream>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecAdd(int *a, int* b, int* c);

int main(){
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], b[N], c[N];
    int *dev_a, *dev_b, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
        fin>>b[i];
    }

    int size = N*sizeof(int);
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size,cudaMemcpyHostToDevice);
    
    vecAdd<<<((N+T-1)/T),T>>>(dev_a,dev_b,dev_c);
    
    cudaMemcpy(c, dev_c, size,cudaMemcpyDeviceToHost);
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nb[]\n";
    for(int i=0;i<N;i++){
        cout<<b[i]<<" ";
    }
    cout<<"\nc[]\n";
    for(int i=0;i<N;i++){
        cout<<c[i]<<" ";
    }
    fin.close();
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecAdd (int *a, int *b, int *c) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i<N){
      c[i] = a[i] + b[i];
    }
}

##Vector Multiply each element by 5

In [ ]:
#multiple block multiple thread, multiply each element by 5 Assign4 Q1
%%cu
#define T 1024
#define N 10000
#include<iostream>
#include <chrono> 
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecMul5(int *a, int* c);

int main(){
    std::chrono::time_point<std::chrono::system_clock> start, end; 
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], c[N];
    int *dev_a, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
    }

    int size = N*sizeof(int);
    start = std::chrono::system_clock::now();
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_c, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    
    vecMul5<<<((N+T-1)/T),T>>>(dev_a,dev_c);
    
    cudaMemcpy(c, dev_c, size,cudaMemcpyDeviceToHost);
    end = std::chrono::system_clock::now();
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nc[]\n";
    for(int i=0;i<N;i++){
        cout<<c[i]<<" ";
    }
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nelapsed time: " << elapsed_seconds.count() << "s\n"; 
    fin.close();
    cudaFree(dev_a);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecMul5 (int *a, int *c) {
    
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i<N){
      c[i] = a[i]*5;
    }
}

##vector pairwise sum


In [ ]:
#multiple block multiple thread, vector pairwise sum by 5 Assign4 Q2
%%cu
#define T 8
#define N 8
#define m -1
#include<iostream>
#include <chrono> 
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecPairSum(int *a, int* c);

int main(){
     std::chrono::time_point<std::chrono::system_clock> start, end; 
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], c[N];
    int *dev_a, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
    }

    int size = N*sizeof(int);
    start = std::chrono::system_clock::now();

    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_c, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    vecPairSum<<<((N+T-1)/T),T>>>(dev_a,dev_c);
    cudaMemcpy(c, dev_c, size,cudaMemcpyDeviceToHost);

    end = std::chrono::system_clock::now();

    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    /*cout<<"\nc[]\n";
    for(int i=0;i<N;i++){
        cout<<c[i]<<" ";
    }*/
    cout<<"\nPairwise sum: "<<c[0];
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nelapsed time: " << elapsed_seconds.count() << "s\n"; 
    fin.close();
    cudaFree(dev_a);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecPairSum (int *a, int *c) {
    int g = threadIdx.x + blockIdx.x * blockDim.x;
    int n = 1;
    if( g < N ) {
        c [ g ] = a [ g ];
        __syncthreads();
        
        while(n<=N/2){
            if(g%n ==0 && ((g/n)%2)!=0){
                c[g-n] += c[g];
            }            
            n = n*2;
        }
    }
    
}

##1 block multiple threads Vector Dot Product

In [ ]:
# 1 block multiple threads, Vector Dot Product Assign4 Q3
%%cu
#define N 1024
#define m -1

#include<iostream>
#include <chrono>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecDotProduct(int *a, int* b, int* c);

int main(){
    std::chrono::time_point<std::chrono::system_clock> start, end; 
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], b[N];
    int *c = (int *)malloc(sizeof(int));
    int *dev_a, *dev_b, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
        fin>>b[i];
    }

    int size = N*sizeof(int);
    start = std::chrono::system_clock::now();
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, sizeof(int));
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size,cudaMemcpyHostToDevice);
    
    vecDotProduct<<<1,N>>>(dev_a,dev_b,dev_c);
    
    cudaMemcpy(c, dev_c, sizeof(int),cudaMemcpyDeviceToHost);
     end = std::chrono::system_clock::now();
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nb[]\n";
    for(int i=0;i<N;i++){
        cout<<b[i]<<" ";
    }
    cout<<"\nDot Product is: "<<c[0];
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nelapsed time: " << elapsed_seconds.count() << "s\n"; 
    fin.close();
    free(c);
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecDotProduct (int *a, int *b, int *c) {
    __shared__ int temp;
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i==0){
        temp=0;
    }
    if(i<N){
      temp+=(a[i]*b[i]);
      __syncthreads();
      
      if(i==(N+m)){
          *c = temp;
      }
    }
}

##Vector Dot Product using multiple blocks, multiple threads

In [ ]:
#Vector Dot Product using multiple blocks multiple threads, Assign 5 Q1
%%cu
#define N 4
#define T 8
#define m -1

#include<iostream>
#include <chrono>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void vecDotProduct(int *a, int* b, int* c);

int main(){
    std::chrono::time_point<std::chrono::system_clock> start, end; 
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N], b[N];
    int *c = (int *)malloc(sizeof(int));
    int *dev_a, *dev_b, *dev_c;
    for(int i = 0;i<N;i++){
        fin>>a[i];
        fin>>b[i];
    }
    int sum=0;
    for(int i=0;i<N;i++){
        sum+=(a[i]*b[i]);
    }
    cout<<sum<<endl;
    int size = N*sizeof(int);
    start = std::chrono::system_clock::now();
    cudaMalloc((void**)&dev_a, size);
    cudaMalloc((void**)&dev_b, size);
    cudaMalloc((void**)&dev_c, sizeof(int));
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, size,cudaMemcpyHostToDevice);
    
    vecDotProduct<<<((N+T-1)/T),T>>>(dev_a,dev_b,dev_c);
    
    cudaMemcpy(c, dev_c, sizeof(int),cudaMemcpyDeviceToHost);
     end = std::chrono::system_clock::now();
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cout<<"\nb[]\n";
    for(int i=0;i<N;i++){
        cout<<b[i]<<" ";
    }
    cout<<"\nDot Product is: "<<c[0];
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nelapsed time: " << elapsed_seconds.count() << "s\n"; 
    fin.close();
    free(c);
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);
    exit (0);
}
__global__ void vecDotProduct (int *a, int *b, int *c) {
    __shared__ int temp[T];
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i==0){
        *c = 0;
    }
    int l = threadIdx.x;
    if(i<N){
        if(l==0){
            temp[l] = a[i] * b[i];
        }
        else{
            temp[l]=temp[l-1]+(a[i]*b[i]);
        }
      if((i==(N+m))){
          *c = *c+temp[l];
      }
      if(l == (blockDim.x+m)){
          *c += temp[l];
      }      
    }
    __syncthreads();
}

##Search the element which is present in first array but not in second array

In [ ]:
#Search the element which is present in first array but not in second array
%%cu
#include<iostream>
#include<stdio.h>
#include<cuda.h>
#include<cmath>
#include<fstream>
using namespace std;

__global__ void search(int* x,int *y,int n)
{
	int id=threadIdx.x;
	int temp;
	if(id<5){
	    temp = x[id];
	    __syncthreads();
	}
	for(int i=0;i<n;i++){
	    if(temp==y[i]){
	        return;
	    }
	}
	printf("%d not present in second array",temp);
}

int main()
{   int a[5] = {1,2,3,4,5};
    int b[5] = {2,3,1,0,5};
    int *dev_a, *dev_b;
    int n = 5;
	cudaMalloc((void**)&dev_a, n*sizeof(int));
	cudaMalloc((void**)&dev_b, n*sizeof(int));
	cudaMemcpy(dev_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
	cudaMemcpy(dev_b,b,n*sizeof(int),cudaMemcpyHostToDevice);
	search<<<1,5>>>(dev_a,dev_b,5);
	cudaMemcpy(a,dev_a,n*sizeof(int), cudaMemcpyDeviceToHost);
	cudaMemcpy(b,dev_b,n*sizeof(int), cudaMemcpyDeviceToHost);
	cudaFree(dev_a);
	cudaFree(dev_b);
	return 0;
}

4 not present in second array


##reverse an array using shared memory

In [ ]:
#reverse an array using shared memory
%%cu
#include<iostream>
#include<cmath>
#include<fstream>
using namespace std;

#define N 10
#define T 1024

__global__ void reverse (int *a) {
    __shared__ int temp[N];
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    __syncthreads();
    if(index < (N/2)){
        temp[index] = a[index];
        a[index] = a[N-index-1];
        a[N-index-1]=temp[index];
    }
}

int main(){
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);

    int a[N];
    int *dev_a;
    cout<<"INTITIAL ARRAY:\n";
    for(int i = 0;i<N;i++){
        fin>>a[i];
        cout<<a[i]<<" ";
    }
    cout<<"\n";

    int size = N*sizeof(int);
    cudaMalloc((void**)&dev_a, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    
    reverse<<<(N+T-1)/T,T>>>(dev_a);
    
    cudaMemcpy(a, dev_a, size,cudaMemcpyDeviceToHost);
    cout<<"\nARRAY AFTER REVERSAL\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    cudaFree(dev_a);

    fin.close();
}

INTITIAL ARRAY:
41 467 334 500 169 724 478 358 962 464 

ARRAY AFTER REVERSAL
464 962 358 478 724 169 500 334 467 41 


##ODD EVEN sort

In [ ]:
#Assign6 Q1, Odd Even Sort
%%cu
#define T 512
#define N 1024
#include<iostream>
#include <chrono> 
#include<cmath>
#include<fstream>
using namespace std;

__global__ void oddeven(int *a, int flag)
{
 int index = blockIdx.x * blockDim.x + threadIdx.x;
  int temp;
  if((index >= N/2 - 1) && flag % 2 != 0) return;

  if(flag % 2 == 0) //if even phase
  {
    if(a[index *2 ] > a[index * 2 + 1])
    {
      temp = a[index * 2];
      a[index * 2] = a[index *2 +1];
      a[index * 2 +1] = temp;
    }
  }
  else { //if odd phase
    if(a[index * 2 +1 ] > a[index *2 + 2])
  {
      temp = a[index * 2 + 1];
      a[index * 2 + 1] = a[index*2+2];
      a[index*2+2] = temp;
    }
  }
}

int main(){
    std::chrono::time_point<std::chrono::system_clock> start, end; 
    ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", ios::in);
    int a[N];
    int *dev_a;
    for(int i = 0;i<N;i++){
        fin>>a[i];
    }

    int size = N*sizeof(int);
    start = std::chrono::system_clock::now();
    cudaMalloc((void**)&dev_a, size);
    
    cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
    
    for(int i=0;i<N;i++)
    {
      oddeven<<<(N+T-1)/T, T>>>(dev_a, i); 
    }
    
    cudaMemcpy(a, dev_a, size, cudaMemcpyDeviceToHost);

    end = std::chrono::system_clock::now();
    cout<<"\na[]\n";
    for(int i=0;i<N;i++){
        cout<<a[i]<<" ";
    }
    
    std::chrono::duration<double> elapsed_seconds = end - start;
    cout<< "\nelapsed time: " << elapsed_seconds.count() << "s\n"; 
    fin.close();
    cudaFree(dev_a);
    exit (0);
}

##Quick sort (host functions)

In [ ]:
#quick sort (host functions)
%%cu

#include <iostream>
#include <fstream>
#include <chrono>

#define T 1024
#define Size 1024

using namespace std::chrono;

__host__ void Swap(int& a, int& b)
{
	int temp = a;
	a = b;
	b = temp;
}

__host__ int Partition(int arr[], int first, int last)
{
	int lastnum = arr[last];
	int i = first - 1;

	for (int j = first; j < last; ++j)
	{
		if (arr[j] < lastnum)
		{
			i++;
			Swap(arr[i], arr[j]);
		}
	}
	Swap(arr[i + 1], arr[last]);

	return (i + 1);
}

__host__ void quicksort(int *data, int first, int last)
{
	int stack[Size], top = -1;
	stack[++top] = first;
	stack[++top] = last;

	while (top >= 0)
	{
		last = stack[top--];
		first = stack[top--];

		int pivot = Partition(data, first, last);

		if (pivot - 1 > first)
		{
			stack[++top] = first;
			stack[++top] = pivot - 1;
		}

		if (pivot + 1 < last)
		{
			stack[++top] = pivot + 1;
			stack[++top] = last;
		}
	}
}

int main()
{
    std::ifstream fin;
    fin.open("/content/drive/My Drive/CSV data/Database.txt", std::ios::in);
    int a[Size];
    int *dev_a;
 
    for(int i = 0; i<Size; ++i)
    {
        fin >> a[i];
    }
    std::cout << "\nUnsorted Array:\n";
    for(int i=0; i<Size; ++i)
    {
        std::cout << a[i] << " ";
    }

    int size = Size*sizeof(int);
    cudaMalloc((void**)&dev_a, size);
 
    auto start = high_resolution_clock::now();
    
    quicksort(a, 0, Size-1);
 
    auto stop = high_resolution_clock::now();
    auto duration = duration_cast<nanoseconds>(stop - start);
 
    std::cout << "\nSorted Array:\n";
    for(int i=0; i<Size; ++i)
    {
        std::cout << a[i] << " ";
    }
    std::cout << "\nTime taken: " << duration.count() << " Nanoseconds" << std::endl;
 
    fin.close();
    exit (0);
}